# common

only unweighted

In [4]:
import networkx as nx
import numpy as np
from utils import load_graph, rodger_graphs, weighted_graphs, unweighted_graphs
from relative_error import spe_relative_error
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
ks = [4]
s = 10
nv = 100

# k

In [7]:
ks = [2,3,4,5,6,7,8,9,10]

In [8]:
def add_relative_error(df, G, ks, s, nv, is_weighted, graph_path=""):
    now_id = 0
    if df.shape[0]>0:
        now_id = df["id"].max()+1
    Gtypes = ["normalized_laplacian", "laplacian", "adjacency"]
    for Gtype in Gtypes:
        errors = spe_relative_error(G, ks, s, nv, Gtype=Gtype, avg_times=10, graph_path=graph_path)
        for i, k in enumerate(ks):
            df = df.append({"id":now_id, "relative_error":errors[i], "k":k, "matrix":Gtype},ignore_index=True)
    return df

df = pd.DataFrame(columns=["id", "relative_error", "k", "matrix"])

# unweighted_graphs
is_weighted = False
for path in unweighted_graphs:
    print("path:", path)
    print("loading graph...")
    G = load_graph(path, is_weighted)
    N = G.number_of_nodes()
    M = G.number_of_edges()
    print(f"N:{N}, M:{M}")
    df = add_relative_error(df, G, ks,s,nv,is_weighted)

# rodger_graphs
is_weighted = False
for path in rodger_graphs:
    print("path:", path)
    print("loading graph...")
    G = load_graph(path, is_weighted)
    N = G.number_of_nodes()
    M = G.number_of_edges()
    print(f"N:{N}, M:{M}")
    df = add_relative_error(df, G, ks,s,nv,is_weighted, graph_path=path)
    
"""
# weighted_graphs
is_weighted = True
for path in weighted_graphs:
    print("path:", path)
    print("loading graph...")
    G = load_graph(path, is_weighted)
    N = G.number_of_nodes()
    M = G.number_of_edges()
    print(f"N:{N}, M:{M}")
    df = add_relative_error(df, G, ks,s,nv,is_weighted)

"""

path: data/networkrepository/bio/bio-celegans/bio-celegans.mtx
loading graph...
N:453, M:2025
path: data/networkrepository/bio/bio-celegans-dir/bio-celegans-dir.edges
loading graph...
N:453, M:2040
path: data/networkrepository/bio/bio-diseasome/bio-diseasome.mtx
loading graph...
N:516, M:1188
path: data/networkrepository/cheminformatics/ENZYMES_g295/ENZYMES_g295.edges
loading graph...
N:123, M:139
path: data/networkrepository/cheminformatics/ENZYMES_g296/ENZYMES_g296.edges
loading graph...
N:125, M:141
path: data/networkrepository/cheminformatics/ENZYMES_g297/ENZYMES_g297.edges
loading graph...
N:121, M:149
path: data/networkrepository/interaction/ia-crime-moreno/ia-crime-moreno.edges
loading graph...
N:829, M:1475
path: data/networkrepository/interaction/ia-email-univ/ia-email-univ.mtx
loading graph...
N:1133, M:5451
path: data/networkrepository/interaction/ia-enron-only/ia-enron-only.mtx
loading graph...
N:143, M:623
path: data/networkrepository/interaction/ia-fb-messages/ia-fb-messa

'\n# weighted_graphs\nis_weighted = True\nfor path in weighted_graphs:\n    print("path:", path)\n    print("loading graph...")\n    G = load_graph(path, is_weighted)\n    N = G.number_of_nodes()\n    M = G.number_of_edges()\n    print(f"N:{N}, M:{M}")\n    df = add_relative_error(df, G, ks,s,nv,is_weighted)\n\n'

In [13]:
df[df["relative_error"]>1000]

,id,relative_error,k,matrix
532,19,452779.922069,3,adjacency
534,19,131363.919965,5,adjacency
536,19,88801.266707,7,adjacency
538,19,66763.265030,9,adjacency
559,20,180550.028178,3,adjacency
561,20,186260.351212,5,adjacency
563,20,188429.870081,7,adjacency
565,20,166684.897047,9,adjacency
586,21,62969.591268,3,adjacency
588,21,46842.595743,5,adjacency


In [10]:
df.groupby("k").mean()

,relative_error
k,
2,0.012699
3,7845.265125
4,0.029152
5,4237.206781
6,0.040049
7,3610.011284
8,0.046878
9,3055.143928
10,0.051888


In [ ]:

ax = sns.lineplot(
    data=fmri,
    x="timepoint", y="signal", hue="event", style="event",
    markers=True, dashes=False
)
ax.set(xlabel="Graph ID",ylabel="rel. error", yscale="log", xticks=[]) #
plt.savefig(f'plot/plot_each_graph_k{k}.png')
plt.show()

# step

In [ ]:
steps = [10, 20, 40, 80, 160, 320, 640, 1280, 2560]

# nv

In [ ]:
nvs = [5, 10, 20, 40, 80, 160]